<p style="font-size:30px">Statistik 2</p>
Hochschule Aalen

Sommersemester 2021

Martin Heckmann

martin.heckmann@hs-aalen.de

21.02.2021

# Assignment 1: Glienvko-Cantelli Theorem 
(central theorem of statistics)

## Target
Target of the assignment is to empricially investigate the central theorem of statistics, i.e. if we independently draw samples from the same distribution the empirical distribution will converge to the true distribution as the number of samples increases

### Task
- Define a normal distribution with mean 50 and variance 5
- Draw the following number of samples from the distribution: 10, 100, 10000, 100000, 1000000, 10000000
- Calculate mean, median and standard deviation of the samples using numpy.mean, numpy.median, numpy.var
- Calculate the empirical distribution from the samples using numpy.histogram, use 21 bins and chose the range meaningfully
- Graphically compare the empirical distributions to the true distribution (as in the example)
- Calculate the squared error between the empirical and true values (error for mean, median, standard deviation and empirical distribution). For the empirical distribution calculate the squared error for each of the bins and calculate the average
- Graphically show the errors for the different number of samples, i.e. make a plot for each statistics (mean, median, std. deviation, histogram) where you show the error in relation to the number of samples used

#### Hints
- The example below shows how some of the tasks can be solved. It is also possible to follow a different approach.
- To define the normal distribution and draw samples from it follow the example below or the descriptions on https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html 
- To calculate the error between the empirical distribution and the true distribution use the pdf function (as in the example) as reference

In [ ]:
# Housekeeping: Loading of necessary packets
import pandas as pd
import numpy as np
from scipy import stats
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
# Fix the random seed to make all experiments reproducible
np.random.seed(1738)

#### Example

In [ ]:

#wird nicht verwendet!
def calcEmpericalArray(arr=None):    
    n=len(arr)
    sort=np.sort(arr)
    ret=[]
    idx=0
    for i in range(n):
        for j in range(idx,n):
            if sort[j]>arr[i]:
                idx=z=j-1
                #print(str(z),end=" ")
                ret.append(z/n)
                break
    print(ret)
    return ret


In [5]:
# Create a normal random variable
#Define a normal distribution with mean 50 and variance 5
mu = 50    #Mittelwert
std = 25    #Standart Abweichung

# Create a normal distribution
norm_rv = stats.norm(loc=mu, scale=std)


#Draw the following number of samples from the distribution: 10, 100, 10000, 100000, 1000000, 10000000
#draws=[10, 100, 10000, 100000, 1000000, 10000000]
draws=[10, 100, 10000, 100000, 1000000]
err_MW=[]
err_MD=[]
err_Var=[]
err_Std=[]
err_Emp=[]
for i in range(len(draws)):
    samples = norm_rv.rvs(size=draws[i])

    #Calculate mean, median and standard deviation of the samples using numpy.mean, numpy.median, numpy.var
    print("\nMittelwert : "+str(np.mean(samples)))
    print("Median     : "+str(np.median(samples)))
    varianz=np.var(samples)
    print("Varianz    : "+str(varianz))
    print("σ          : "+str(np.sqrt(varianz))+" (Standardabweichung)")

    #empirical=calcEmpericalArray(samples)
    #print(empirical)
    print(len(samples))
    print(samples)
    range_min = 0
    range_max = 100
    nb_bins = 21
    # Create x range to plot the normal distribution
    x = np.linspace(range_min, range_max, 100)
    #print(x)
    #bins sind die Anzahl der Säulen!
    # Calculate histogram with absolute frequencies
    samples_hist, bin_edges = np.histogram(samples,
        bins=nb_bins,range=(range_min,range_max),  density=True)#range=(range_min, range_max)
    
    
    # Convert the values of the edges to their centers (needed for plotting)
    bin_centers = (bin_edges[:-1]+bin_edges[1:])/2

    # Put histogram in a data frame for plotting
    histVals_df = pd.DataFrame(np.column_stack(
        (bin_centers, samples_hist)), columns=['Samples', 'Rel. Frequency'])

    # Plot the histogram
    fig = px.bar(histVals_df, x="Samples", y=["Rel. Frequency"], barmode="overlay")


    pdfed=norm_rv.pdf(x)
    fig.add_scatter(x=x, y=pdfed, mode="lines", line=dict(color="gray"), showlegend = False)
    #empirical=calcEmpericalArray(samples_hist)
    #fig.add_scatter(x=np.arange(40,40+len(empirical)), y=empirical, mode="lines", line=dict(color="red"), showlegend = False)
    #fig.add_scatter(x=x, y=norm_rv.pdf(x), mode="lines", line=dict(color="gray"), showlegend = False)
    fig.update_layout(
        yaxis=dict(title="Empirical density",
            range=[numpy.amin(),.1]
        ),
        font=dict(
            family="Helvetica",
            size=16,
            color="Black"
        ),
        showlegend=False,
        autosize=False,
        width=600,
        height=500,
        margin=dict(
            l=10,
            r=10,
            b=10,
            t=50,
            pad=4
        ),
    )
    fig.show()
    print("Sample Vergleich Quadratischer Fehler: ")
    z1=np.mean(samples_hist)
    z2=np.mean(pdfed)
    erg=(z1-z2)**2
    err_MW.append(erg)
    print("\nMittelwert : "+str(erg)+"\t-> "+str(z1)+" - "+str(z2))
    z1=np.median(samples_hist)
    z2=np.median(pdfed)
    erg=(z1-z2)**2
    err_MD.append(erg)
    print("Median     : "+str(erg)+"\t-> "+str(z1)+" - "+str(z2))
    z1=np.var(samples_hist)
    z2=np.var(pdfed)
    erg=(z1-z2)**2
    err_Var.append(erg)
    print("Varianz    : "+str(erg)+"\t-> "+str(z1)+" - "+str(z2))
    z1=np.sqrt(np.var(samples_hist))
    z2=np.sqrt(np.var(pdfed))
    erg=(z1-z2)**2
    err_Std.append(erg)
    print("σ          : "+str(erg)+' Standardabweichung'+"\t-> "+str(z1)+" - "+str(z2))
    #emperical error:
    bins_err=[]
    for j in range(1,nb_bins+1):
        #print(j,end=" ")
        z=(int)(np.round(j*len(x)/nb_bins))
        
        bins_err.append((samples_hist[j-1]-pdfed[z-1])**2)
    erg=np.mean(bins_err)
    err_Emp.append(erg)
    print("Emperisch  : "+str(erg))
    print()
    
    print(str(len(samples_hist))+" - "+str(len(pdfed)))
    #print(bin_centers)

#mache Error VergleichsGraph
#x -> draws
#y -> Quadratischer fehler
str_draws=[]
for i in range(len(draws)):
    str_draws.append(str(draws[i]))
df = pd.DataFrame({
    "draws": str_draws,
    "Mittelwert": err_MW,
    "Median": err_MD,
    "Varianz": err_Var,
    "Standardabweichung": err_Std,
    "Emperisch": err_Emp
})
fig = px.bar(df, x="draws", y=["Mittelwert","Median","Varianz","Standardabweichung","Emperisch"], barmode="group",log_y=True,title="LOG Quad. Error")
fig.show()
fig2 = px.bar(df, x="draws", y=["Mittelwert","Median","Varianz","Standardabweichung","Emperisch"], barmode="group",log_y=False,title="Linear Quad. Error")
fig2.show()
print("fin")

NameError: name 'stats' is not defined

In [14]:
import scipy.stats
import numpy
import pandas
import plotly.express
import plotly.graph_objs

# Fix the random seed to make all experiments reproducible
numpy.random.seed(1738)

Mittelwert=50
Varianz=5
Saeulen=21
ProzentSchranke=0.999
Normalverteilung=scipy.stats.norm(loc=Mittelwert,scale=numpy.sqrt(Varianz))
range_min,range_max=Normalverteilung.interval(ProzentSchranke)

print("Normalverteilung (scipy.stats.norm):\n")
print("Standardabweichung: "+str(Normalverteilung.std()))
print("Varianz           : "+str(Normalverteilung.var()))
print("Mittelwert        : "+str(Normalverteilung.mean()))
print("Median            : "+str(Normalverteilung.median()))
print("Unteres x von "+str(ProzentSchranke*100)+"% der Werte: "+str(range_min))
print("Oberes  x von "+str(ProzentSchranke*100)+"% der Werte: "+str(range_max))
print("\n")
#100 X Werte zwischen range_min & range_max:
x=numpy.linspace(range_min,range_max,100)
#Wahrscheinlichkeitsdichtefunktion/probability density function
#für die XKoordinaten von x
pdf=Normalverteilung.pdf(x)
#ErrorWert Listen:
means,medians,stds,emps=[],[],[],[]
draws=[10, 100, 10000, 100000, 1000000, 10000000]
for m,size in enumerate(draws):
    print("m="+str(m)+" size="+str(size))
    #Zufällige Werte aus der Verteilung ziehen
    sample=Normalverteilung.rvs(size=size)
    #print(sample[0])#gibt ein Zufälligen Wert aus der aus der Normalverteilung entnommen wurde
    YHistogram,XHistogram=numpy.histogram(sample,bins=Saeulen,range=(range_min,range_max),density=True)
    #print(YHistogram)#Relative Häufigkeiten der Samples
    #Eine Liste mit den XKoordinaten die in der Mitte der Balken sind!
    XCenterH=(XHistogram[:-1]+XHistogram[1:])/2    
    #print(len(XCenterH))# dasselbe wie die Anzahl der Säulen natürlich!
    #X hat ein X mehr weil ein Balken hat 2 Punkte -> 21 Balken -> 1.Balken 2, die Restlichen 1 Punkt also 22 Punkte
    #print(str(len(YHistogram))+" / "+str(len(XHistogram)))
    
    #Balkendiagramm 
    df=pandas.DataFrame(numpy.column_stack((XCenterH,YHistogram)),columns=["Samples","Relative Häufigkeit"])
    figure=plotly.express.bar(df,x="Samples",y="Relative Häufigkeit")
    
    #"Normalverteilungskurve"
    figure.add_scatter(x=x,y=pdf,mode="lines",showlegend=False,line=dict(color="blue"))
    figure.show()
    
    #Quadratischen Fehler merken:
    means.append((Normalverteilung.mean()-sample.mean())**2)
    medians.append((Normalverteilung.median()-sample.mean())**2)
    stds.append((Normalverteilung.std()-sample.std())**2)
    emp_errors=[]
    for i in range(Saeulen):
        #Der "richtige Y Wert", der Y Wert der Wahrscheinlichkeitsdichtefunktion an Stelle des Balken
        WTrueValue=Normalverteilung.pdf(x=XCenterH[i])
        emp_errors.append((WTrueValue-YHistogram[i])**2)
    emps.append(numpy.mean(emp_errors))    
    print("Standardabweichung: "+str(sample.std()))
    print("Varianz           : "+str(sample.var()))
    print("Mittelwert        : "+str(sample.mean()))
    print("Median            : "+str(numpy.median(sample)))
    print()
    print(f"Mean squared error     : {means[m]}")
    print(f"Median squared error   : {medians[m]}")
    print(f"Std squared error      : {stds[m]}")
    print(f"Empirical squared error: {emps[m]}")
    
    print("------------------------------------------------\n")
    

Normalverteilung (scipy.stats.norm):

Standardabweichung: 2.23606797749979
Varianz           : 5.000000000000001
Mittelwert        : 50.0
Median            : 50.0
Unteres x von 99.9% der Werte: 42.64215854660392
Oberes  x von 99.9% der Werte: 57.35784145339615


m=0 size=10


Standardabweichung: 2.9470649267176485
Varianz           : 8.685191682289298
Mittelwert        : 50.91126448946194
Median            : 51.33947881520042

Mean squared error     : 0.8304029697543341
Median squared error   : 0.8304029697543341
Std squared error      : 0.5055166617971024
Empirical squared error: 0.010843392296052105
------------------------------------------------

m=1 size=100


Standardabweichung: 2.282750235252523
Varianz           : 5.2109486365454485
Mittelwert        : 49.62967181937214
Median            : 49.83899431810238

Mean squared error     : 0.13714296136713958
Median squared error   : 0.13714296136713958
Std squared error      : 0.0021792331888926047
Empirical squared error: 0.0007399626659559571
------------------------------------------------

m=2 size=10000


Standardabweichung: 2.2316834442812388
Varianz           : 4.980410995478972
Mittelwert        : 49.96306911470527
Median            : 49.927414675652344

Mean squared error     : 0.0013638902886524953
Median squared error   : 0.0013638902886524953
Std squared error      : 1.922413154457749e-05
Empirical squared error: 1.4875499599031339e-05
------------------------------------------------

m=3 size=100000


Standardabweichung: 2.241841366053207
Varianz           : 5.025852710547308
Mittelwert        : 49.99262307343392
Median            : 49.985078140761004

Mean squared error     : 5.441904556135848e-05
Median squared error   : 5.441904556135848e-05
Std squared error      : 3.333201538872673e-05
Empirical squared error: 1.1596916592218896e-06
------------------------------------------------

m=4 size=1000000


Standardabweichung: 2.2377446612752325
Varianz           : 5.007501169065805
Mittelwert        : 49.99592709883815
Median            : 49.993562530584526

Mean squared error     : 1.658852387418367e-05
Median squared error   : 1.658852387418367e-05
Std squared error      : 2.8112684828328607e-06
Empirical squared error: 2.8084135703771464e-07
------------------------------------------------

m=5 size=10000000


Standardabweichung: 2.236908188272475
Varianz           : 5.003758242760446
Mittelwert        : 50.00018183127106
Median            : 49.999450829492815

Mean squared error     : 3.306261113415504e-08
Median squared error   : 3.306261113415504e-08
Std squared error      : 7.059541425363566e-07
Empirical squared error: 1.0957550163356093e-07
------------------------------------------------



In [15]:
#erstelle Datafram wie Matthias Pijarowski 
err_df=pandas.DataFrame()
err_df["samples"]=draws
err_df['mean_error'] =  means
err_df['median_error'] = medians
err_df['std_error'] = stds
err_df['emp_error'] = emps
#Dataframe anzeigen
err_df
#print(err_df)

,samples,mean_error,median_error,std_error,emp_error
0,10,8.304030e-01,8.304030e-01,5.055167e-01,1.084339e-02
1,100,1.371430e-01,1.371430e-01,2.179233e-03,7.399627e-04
2,10000,1.363890e-03,1.363890e-03,1.922413e-05,1.487550e-05
3,100000,5.441905e-05,5.441905e-05,3.333202e-05,1.159692e-06
4,1000000,1.658852e-05,1.658852e-05,2.811268e-06,2.808414e-07
5,10000000,3.306261e-08,3.306261e-08,7.059541e-07,1.095755e-07


In [16]:
#Anzeige von Roman
str_draws=[]
for i in range(len(draws)):
    str_draws.append(str(draws[i]))
err_df2=err_df
err_df2["samples"]=str_draws

fig = plotly.express.bar(err_df2, x="samples", y=["mean_error","median_error","std_error","emp_error"], barmode="group",log_y=True,title="LOG Quad. Error")
fig.show()
fig2 = plotly.express.bar(err_df2, x="samples", y=["mean_error","median_error","std_error","emp_error"], barmode="group",log_y=False,title="Linear Quad. Error")
fig2.show()



In [4]:
#Matthias Pijarowski Graphen:
for colname in ["mean_error", "median_error", "std_error", "emp_error"]:
    fig = plotly.graph_objs.Figure()
    fig.add_trace(plotly.graph_objs.Scatter(y=err_df[colname], x=err_df['samples'], mode='lines+markers'))
    fig.update_layout(xaxis_type='category',
                      xaxis_title="Samples",
                      yaxis_title=f"{colname} squared",
                      margin=dict(
                          pad=50
                      ),
                 )
    fig.show()